qid1 and qid2 :bow
is_duplicate :ml algo(for prediction 'y')

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.shape

(404290, 6)

In [4]:
df.sample(5)

,id,qid1,qid2,question1,question2,is_duplicate
260732,260732,11310,376731,Dating and Relationships: I am horny often and...,Time 2.15 am. But I can not sleep . I don't kn...,0
171873,171873,265511,235732,Who all are in the technical team at Zomato?,What does the Zomato logo mean?,0
90795,90795,152303,152304,I was given an assignment of 'In situ ex situ ...,I have got 85% in my 12th and I'm a non- mahar...,0
214627,214627,320268,320269,What does it mean if a girl tells a guy that s...,If a girl keeps replying to a guy will she see...,0
251925,251925,331842,127325,"Which single fact, discovery, document, or tes...","Which single fact, made public before the inau...",1


In [6]:
new_df= df.sample(30000) #since large dataset to minimize the effect

In [9]:
# missing values
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [8]:
new_df.duplicated().sum()

0

In [10]:
ques_df = new_df[['question1','question2']]
ques_df.sample(5)

,question1,question2
349835,How can a beginner learn how to do splits in o...,How can I learn to do splits in one day?
172851,How do I really make money online?,What is the best way for making money online?
55917,What does the symbol ~ mean?,What do these symbols mean?
123166,Do I need a lawyer prepared NDA when talking t...,I get bored very easily when working on the sa...
303424,How many films has Adam Sandler and Rob Schnei...,In which movie did we see peak Adam Sandler?


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
#merge text
questions = list(ques_df['question1']) + list(ques_df['question2']) #list of all questions

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2) #bow for every question

In [14]:
temp_df1 = pd.DataFrame(q1_arr, index= ques_df.index) #into dataframe
temp_df2 = pd.DataFrame(q2_arr, index= ques_df.index)
temp_df = pd.concat([temp_df1,temp_df2], axis=1) #and concatinate them
temp_df.shape # 6000 features since 3000 for question1 and 3000 for question2

(30000, 6000)

In [17]:
temp_df

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
182379,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
278247,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
333868,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
192270,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
207659,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288212,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38910,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100191,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
182956,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
temp_df['is_duplicate']= new_df['is_duplicate']

In [16]:
temp_df.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
377929,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
335271,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
202342,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
347183,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
167105,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(temp_df.iloc[:,0:-1].values, temp_df.iloc[:,-1].values, test_size=0.2, random_state=42)

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.7416666666666667

In [21]:
pip install xgboost



   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/99.8 MB 1.3 MB/s eta 0:01:18
   ---------------------------------------- 0.1/99.8 MB 1.3 MB/s eta 0:01:16
   ---------------------------------------- 0.1/99.8 MB 798.9 kB/s eta 0:02:05
   ---------------------------------------- 0.2/99.8 MB 1.1 MB/s eta 0:01:33
   ---------------------------------------- 0.3/99.8 MB 1.3 MB/s eta 0:01:19
   ---------------------------------------- 0.4/99.8 MB 1.5 MB/s eta 0:01:07
   ---------------------------------------- 0.5/99.8 MB 1.5 MB/s eta 0:01:05
   ---------------------------------------- 0.6/99.8 MB 1.7 MB/s eta 0:00:59
   ---------------------------------------- 0.7/99.8 MB 1.6 MB/s eta 0:01:02
   ---------------------------------------- 0.7/99.8 MB 1.6 MB/s eta 0:01:02
   ---------------------------------------- 0.9/99.8 MB 1.7 MB/s eta 0:00:59
   ---------------------------------------- 1.0/99.8 MB 1.7 MB/s eta 0:00:58
   

In [24]:
pip install xgboost scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [26]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Assuming X_train, X_test, y_train, y_test are already defined and prepared

# Initialize XGBClassifier
xgb = XGBClassifier()

# Fit the model on training data
xgb.fit(X_train, y_train)

# Predict on the test data
y_pred = xgb.predict(X_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.7258333333333333
